In [1]:
import pandas as pd
import numpy as np
#для парсинга сайта
import requests
import bs4
#для препроцессинга текста
from pymystem3 import Mystem
from string import punctuation
#для векторизация текста
from sklearn.feature_extraction.text import CountVectorizer
#для создания модели классификации
from sklearn.linear_model import LogisticRegression
#для 7 недели выгрузка векторизатора и модели через pickle
import pickle

### 1. Загрузим тестовый датасет

In [2]:
# считаем содержимое файла
with open('test.csv', 'r') as f:
   s =(f.read())
    

In [3]:
# разобьем на части с помощью разделителя в нестандартной конфигурации
s = s.split('</review>\n\n<review>')

In [4]:
# для комфорта линчого помещаю в датафрейм
df = pd.DataFrame(s, columns=['review'])

In [5]:
# удалю остатки тегов, символы переноса строки и лишние пробелы - легкий препроцессинг тестовый выборки
df.review = df.review.str.replace('<review>','').str.replace('</review>','').str.replace('\n',' ').str.replace('\s+', ' ', regex=True).str.lower()

In [6]:
#тестовый датасет подготовлен и не содержит тегов и символа переноса строки
df.review[0]

'ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. качества звука через динамик далеко не на высоте.наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти. несколько неудобно что нету отдельной кнопки для фото. подумываю купить батарею большей емкость мб что нибудь измениться. '

### 2. Подготовим обучающую выборку с помощью request и BS4

In [7]:
#версия подготовки со Scrapy обучающей выборки в моей реализации представляет собой папку с кучей файлов и решил для тетрадки остановиться на более простых и понятных request+BS4

#### 2.1 Сначала подготовлю список страниц, которые буду парсить - каждая страница это набор отзывов по телефону

In [8]:
# нагуглил сайт, в котором есть отзывы по телефонам с разметкой
req = requests.get('https://peredpokupkoy.ru/category/elektronika/telefony/mobilnye-telefony/')

In [9]:
soup = bs4.BeautifulSoup(req.text, 'lxml')

In [10]:
# выгружу страницы с отзывами с первой страницы
pages = list()
for el in soup.find('div', attrs={'class':'category-products'}).findAll('a'):
    pages.append(el.get('href'))

In [11]:
#вытащим все страницы с отзывами через пагинацию - диапозон 2-217 взят из личного осмотра пагинация сайта
for i in range(2,217):
    req = requests.get(f'https://peredpokupkoy.ru/category/elektronika/telefony/mobilnye-telefony/page/{i}/')
    soup = bs4.BeautifulSoup(req.text, 'lxml')
    for el in soup.find('div', attrs={'class':'category-products'}).findAll('a'):
        pages.append(el.get('href'))

In [12]:
# удалю дупликаты
pages = set(pages)

In [13]:
# и обратно в список для удобства
pages = list(pages)

#### 2.2 Получу непосредственно отзывы. 

In [14]:
#теперь у меня есть 5000+ страниц с отзывами. Каждая страница содержит отзывы на конкретную модель телефона

In [15]:
# функция, которая собирает отзывы со страницы
def get_reviews(pages):
    # создам словарь куда буду добавлять отзывы со страниц page in pages
    vyborka = {'phone':[], 'dignities':[], 'limitations':[], 'comments':[], 'rating':[]}
    for page in pages:
        req = requests.get('https:' + page)
        soup = bs4.BeautifulSoup(req.text, 'lxml')
        # добавляем рейтинги
        for el in soup.findAll('meta', attrs={'itemprop':'ratingValue'}):
            vyborka['rating'].append(el.get('content'))
            #заоодно заполним данные по телефону, тут мы отталкиваемся от того, рейтинг есть во всех отзывах
            vyborka['phone'].append(req.url[32:].replace('/',''))
        
        for el in soup.findAll('div', attrs={'class':'cm_single_comment_content'}):
            #добавляем достоинства
            if el.find('div', attrs={'class':'cm_single_comment_dignity_ins'})==None:
                vyborka['dignities'].append('')
            else:
                vyborka['dignities'].append(el.find('div', attrs={'class':'cm_single_comment_dignity_ins'}).text)
            #добавляем недостатки
            if el.find('div', attrs={'class':'cm_single_comment_limitations_ins'})==None:
                vyborka['limitations'].append('')
            else:
                vyborka['limitations'].append(el.find('div', attrs={'class':'cm_single_comment_limitations_ins'}).text)
            #добавляем комментарий/вывод
            if el.find('div', attrs={'class':'cm_single_comment_message_ins'})==None:
                vyborka['comments'].append('')
            else:
                vyborka['comments'].append(el.find('div', attrs={'class':'cm_single_comment_message_ins'}).text)       
    return vyborka

In [140]:
# опытным путем определил чисто страниц для парсинга так, чтобы на 1 класс было не меньше 5000 примеров в нашей базе для обучения
# Эта ячейка собирает отзывы с 900 страниц и это без использования параллельных потоков - ДОЛГО, заварите чаек
res = get_reviews(pages[:900])

In [142]:
#результат помещаю в датафрейм
df_vyb=pd.DataFrame(res)

In [144]:
#df_vyb.to_csv('train_dirt.csv')

In [16]:
df_vyb = pd.read_csv('train_dirt.csv', index_col=0)

In [17]:
df_vyb.shape

(19703, 5)

In [18]:
df_vyb.head()

,phone,dignities,limitations,comments,rating
0,vertex-impress-mars-1713741426,Большой экран. Хорошая матрица. Аккумулятор то...,NaN,Идеальный смартфон до 100$. Много перебрал пер...,5
1,vertex-impress-mars-1713741426,все норм,при разговоре постоянный шум микрофона,NaN,3
2,vertex-impress-mars-1713741426,"Цена, камера, 2 симки","Глючит всё, что только может глючить. Сенсорны...",Не связывайтесь с киайфонами!,1
3,vertex-impress-mars-1713741426,"Экран, скорость, качество сборки, дизайн.",Пока не выявлено,"В общем телефон достойный,большой яркий экран,...",5
4,rover-pc-c6,полноценный коммуникатор за относительно небол...,"аккумулятор очень слабый, китайский, в общем д...","В общем неплохая штуковина, с поправкой на акк...",3


In [19]:
#cхлопну отзыв состоящий из трех частей Достоинства, Недостатки и Вывод в одну длинную строку
df_vyb['review'] = df_vyb.dignities.astype('str').str.replace('nan','').str.lower()+' '+df_vyb.limitations.astype('str').str.replace('nan','').str.lower()+' '+df_vyb.comments.astype('str').str.replace('NaN','').str.lower()

In [20]:
# у нас должно быть два класса, а тут 5 классов - выполню маппинг для сокращения классов
df_vyb['rating_binar'] = df_vyb['rating'].map({1:'neg', 2:'neg', 3:'neg',4:'pos',5:'pos'})

In [21]:
# моделям обычно проще работать с числами - еще маппинг
df_vyb['y'] = df_vyb['rating_binar'].map({'neg':0,'pos':1})

In [22]:
#колиество объекто для каждого класса больше 5000, значит выборки достаточно для baseline моделирования
df_vyb['y'].value_counts()

1    13777
0     5926
Name: y, dtype: int64

In [23]:
# выведу один обзор из обучающей выборки - видно, что нужен препроцессинг для обучения - много разных форм слов
df_vyb['review'][0]

'большой экран. хорошая матрица. аккумулятор тоже большой. 13мп камера делает довольно неплохие снимки. быстрый интернет через 4g хорошо пашет gps  идеальный смартфон до 100$. много перебрал перед покупкой, решил довериться неизвестной мне до этого фирме, не пожалел.'

### 3. Препроцессинг

как препроцессить русский текст в 2020:
* https://habr.com/ru/post/503420/
* https://www.kaggle.com/alxmamaev/how-to-easy-preprocess-russian-text

In [24]:
#Лемматейзер
mystem = Mystem() 

Installing mystem to /Users/ltorrick/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [25]:
#функция препроцессинга текста. Не стал использовать стопслова, для сентимент анализа не исключал стопслова
#-это ухудшает часто качество в подобных классификациях, тут на линейной моделе тоже мешали
def preprocess_text_wo_stop(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [26]:
# пробую на 1 отзыве - шикарно
preprocess_text_wo_stop(df_vyb['review'][0])

'большой экран хороший матрица аккумулятор тоже большой 13мп камера делать довольно неплохой снимок быстрый интернет через 4g хорошо пахать gps идеальный смартфон до 100 много перебирать перед покупка решать доверяться неизвестный я до это фирма не пожалеть'

In [27]:
# было до
df_vyb['review'][0]

'большой экран. хорошая матрица. аккумулятор тоже большой. 13мп камера делает довольно неплохие снимки. быстрый интернет через 4g хорошо пашет gps  идеальный смартфон до 100$. много перебрал перед покупкой, решил довериться неизвестной мне до этого фирме, не пожалел.'

In [28]:
#препроцессю всю обучающую выборку - это занимает некоторое время
df_vyb['review_pr_wo_stop'] = df_vyb['review'].apply(preprocess_text_wo_stop)

### 4. Векторизация и обучение модели классификации

In [29]:
# ввожу векторизатор
bow_vectorizer = CountVectorizer(ngram_range=(1,2))

In [30]:
# векторизую препроцешенный текст
X_train_wo_stop = bow_vectorizer.fit_transform(df_vyb['review_pr_wo_stop'] )

In [31]:
y_train = df_vyb.y.values

In [32]:
#sparce матрица объемная и разряженная, чтобы lr не ругался увеличу число итераций
#пробовал SVC и Байес из scklear, на таком обучающем множестве они похуже справлялись, 
#а LR без серьезных инвестиций времени в подбор параметров выдал на CV необходимое качество 0.85+ при 0.97 на тесте (что связываю с тем,
#что возможно на том же сайте я ненамеренно могу получить и тестовые данные)
lr = LogisticRegression(max_iter=10000)

In [33]:
#займет некоторое время
lr.fit(X_train_wo_stop, y_train)

LogisticRegression(max_iter=10000)

### 5. Классификация тестовой выборки

In [34]:
df['review_pr'] = df['review'].apply(preprocess_text_wo_stop)

In [35]:
res = lr.predict(bow_vectorizer.transform(df['review_pr'] ))

In [36]:
df['pred_digit'] = res

In [37]:
df['y']=df['pred_digit'].map({0:'neg', 1:'pos'})

In [38]:
df.index.name = 'Id'

In [42]:
df[['y']].to_csv('first_try.csv')

### 6. Сделаем пару pickle для недели 7

In [55]:
#выгрузим векторайзер
pickle.dump(bow_vectorizer,open('bow_vectorizer.pickle','wb'))


In [56]:
# выгрузим модель
pickle.dump(lr,open('model.pickle','wb'))